In [1]:
import tensorflow as tf
import scipy.io
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mat_train = scipy.io.loadmat('/Users/babburi/Desktop/train/cropped/train_32x32.mat')
mat_test = scipy.io.loadmat('/Users/babburi/Desktop/test/cropped/test_32x32.mat')
X_train = np.array(mat_train['X'])
y_train = np.array(mat_train['y'])
X_test = np.array(mat_test['X'])
y_test = np.array(mat_test['y'])
# replace 10 with 0 in y_train and y_test
y_train[y_train==10] = 0
y_test[y_test==10] = 0
y_train = np.eye(10)[y_train.astype('int')]
y_train = np.reshape(y_train, [y_train.shape[0], 10])
y_test = np.eye(10)[y_test.astype('int')]
y_test = np.reshape(y_test, [y_test.shape[0], 10])
del mat_train, mat_test

In [3]:
batch_size = 256
n_classes = 10

# tf graph input
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32, [None, n_classes])

# convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

# pooling
def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# build cnn model
def convolutional_neural_network(x):
    weights = {
        # 5x5 convolution, 1 input image, 32 outputs
        'W_conv1': tf.Variable(tf.truncated_normal([5,5,3,32], stddev=0.1)),
        
        # 5x5 convolution, 32 inputs, 64 outputs
        'W_conv2': tf.Variable(tf.truncated_normal([5,5,32,64], stddev=0.1)),
        
        # fully connected layer, 8x8x64 inputs, 1024 outputs
        'W_fc': tf.Variable(tf.truncated_normal([8*8*64,1024], stddev=0.1)),
        
        # 1024 inputs, 10 outputs (class prediction)
        'out': tf.Variable(tf.truncated_normal([1024, n_classes], stddev=0.1))
    }
    
    biases = {
        'b_conv1': tf.Variable(tf.constant(0.1, shape=[32])),
        'b_conv2': tf.Variable(tf.constant(0.1, shape=[64])),
        'b_fc': tf.Variable(tf.constant(0.1, shape=[1024])),
        'out': tf.Variable(tf.constant(0.1, shape=[n_classes]))
    }
    
    # reshape input to a 4D tensor
    x = tf.reshape(x, shape=[-1,32,32,3])
    
    # convolution layer 1 followed by pooling
    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    # convolution layer 2 followed by pooling
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)
    
    # fully connected layer
    fc = tf.reshape(conv2, shape=[-1, 8*8*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc']) + biases['b_fc'])
    
    # output layer
    output = tf.matmul(fc, weights['out']) + biases['out']
    return output

In [8]:
def train_cnn(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
    correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            epoch_loss = 0
            i = 0
            while i <= X_train.shape[3]:
                start = i
                end = i + batch_size
                batch_x = X_train[:,:,:,start:end]
                batch_y = y_train[start:end,:]
                
                _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x,
                                                                y:batch_y})
                epoch_loss += c
                i += batch_size
            print('Epoch', epoch+1, 'completed out of', epochs, 'loss:', epoch_loss)
            randsel = np.random.randint(1, X_train.shape[3], batch_size)
            train_accuracy = accuracy.eval(feed_dict={x:X_train[:,:,:,randsel], y:y_train[randsel,:]})
            print("Training accuracy %g", train_accuracy)
        
        print('Test Accuracy:', accuracy.eval({x:X_test, y:y_test}))



In [9]:
train_cnn(x)

Epoch 1 completed out of 10 loss: 230255886566.0
Training accuracy %g 0.15625
Epoch 2 completed out of 10 loss: 642.743298054
Training accuracy %g 0.136719
Epoch 3 completed out of 10 loss: 641.98743701
Training accuracy %g 0.171875
Epoch 4 completed out of 10 loss: 641.922597647
Training accuracy %g 0.199219
Epoch 5 completed out of 10 loss: 641.915437222
Training accuracy %g 0.179688
Epoch 6 completed out of 10 loss: 641.914470673
Training accuracy %g 0.183594
Epoch 7 completed out of 10 loss: 641.914350271
Training accuracy %g 0.183594
Epoch 8 completed out of 10 loss: 641.91437149
Training accuracy %g 0.214844
Epoch 9 completed out of 10 loss: 641.91438961
Training accuracy %g 0.238281
Epoch 10 completed out of 10 loss: 641.914414167
Training accuracy %g 0.207031
Test Accuracy: 0.195874
